## Импорт из PostgreSQL в Google Sheets

Импортируем библиотеки

In [1]:
import requests
import pandas as pd
import psycopg2
from psycopg2.extras import DictCursor
import gspread
from oauth2client.service_account import ServiceAccountCredentials

Первая функция подключается к БД и собирает названия столбцов из таблицы

In [23]:
def make_columns_names():
    columns = []
    with psycopg2.connect(dbname='super_store_db1', user='XXX', 
                        password='XXX', host='XXX.eu-central-1.rds.amazonaws.com') as conn:
        with conn.cursor(cursor_factory=DictCursor) as cursor:        
            cursor.execute("SELECT *FROM information_schema.columns WHERE table_schema = 'rxl' AND table_name = 'points';")
            for row in cursor:
                columns.append(row[3])
    print(columns)                    

In [34]:
make_columns_names()

['pid', 'incomingtrackid', 'speed', 'latitude', 'longitude', 'pointdate', 'height', 'acceleration', 'deceleration', 'ticktimestamp', 'accelerationxoriginal', 'accelerationyoriginal', 'accelerationzoriginal']


Вторая функция создает промежуточные csv-файлы с содержимым таблицы-источника
и затем импортирует данные из них в Google Sheets

In [4]:
def run_import():
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('XXXXXXXXX.json', scope)
    gc = gspread.authorize(creds)
    with psycopg2.connect(dbname='super_store_db1', user='XXX', 
                            password='XXX', host='XXXXX.eu-central-1.rds.amazonaws.com') as conn:
        with conn.cursor(cursor_factory=DictCursor) as cursor:
            cursor.execute('select * from rxl.points limit 5000;')
            for row in cursor:
                df_points = pd.DataFrame(cursor, columns=columns)
            df_points.to_csv('points_psg.csv', index=False)
            content = open('points_psg.csv', 'r').read()
            gc.import_csv('XXX-dmc', content.encode('utf-8')) 

In [ ]:
# Запускаем всю связку
def run_all():
    make_columns_names()
    run_import()

### Версия скрипта для трех таблиц

In [25]:
'''версия для трех таблиц'''
def make_columns_names_x3():
    columns_p = []
    columns_t = []
    columns_o = []
    with psycopg2.connect(dbname='super_store_db1', user='XXX', 
                        password='XXX', host='XXXXX.eu-central-1.rds.amazonaws.com') as conn:
        with conn.cursor(cursor_factory=DictCursor) as cursor:        
            cursor.execute("SELECT *FROM information_schema.columns WHERE table_schema = 'rxl' AND table_name = 'points';")
            for row in cursor:
                columns_p.append(row[3])
        with conn.cursor(cursor_factory=DictCursor) as cursor:        
            cursor.execute("SELECT *FROM information_schema.columns WHERE table_schema = 'rxl' AND table_name = 'tracks';")
            for row in cursor:
                columns_t.append(row[3])
        with conn.cursor(cursor_factory=DictCursor) as cursor:        
            cursor.execute("SELECT *FROM information_schema.columns WHERE table_schema = 'rxl' AND table_name = 'os';")
            for row in cursor:
                columns_o.append(row[3])                       

In [38]:
def run_import_x3():
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('XXXX.json', scope)
    gc = gspread.authorize(creds)
    with psycopg2.connect(dbname='super_store_db1', user='XXX', 
                            password='XXX', host='XXXXX.eu-central-1.rds.amazonaws.com') as conn:
        with conn.cursor(cursor_factory=DictCursor) as cursor:
            cursor.execute('select * from rxl.points limit 5000;')
            for row in cursor:
                df_points = pd.DataFrame(cursor, columns=columns_p)
            df_points.to_csv('points_psg.csv', index=False)
            content = open('points_psg.csv', 'r', encoding="utf-8").read()
            gc.import_csv('XXX-dmc', content.encode('utf-8'))
        with conn.cursor(cursor_factory=DictCursor) as cursor:
            cursor.execute('select * from rxl.tracks;')
            for row in cursor:
                df_tracks = pd.DataFrame(cursor, columns=columns_t)
            df_tracks.to_csv('tracks_psg.csv', index=False)
            content = open('tracks_psg.csv', 'r', encoding="utf-8").read()
            gc.import_csv('XXX', content.encode('utf-8'))
        with conn.cursor(cursor_factory=DictCursor) as cursor:
            cursor.execute('select * from rxl.os;')
            for row in cursor:
                df_os = pd.DataFrame(cursor, columns=columns_o)
            df_os.to_csv('os_psg.csv', index=False)
            content = open('os_psg.csv', 'r').read()
            gc.import_csv('XXX', content.encode('utf-8'))      

In [43]:
from datetime import datetime
import time
import schedule
schedule.every().day.at("12:02").do(run_all).tag('daily_import_sheets')

# при запуске цикла job становится сразу же на мониторинг
#while True:
    #schedule.run_pending()

Every 1 day at 12:02:00 do run_all() (last run: [never], next run: 2020-11-27 12:02:00)

In [49]:
# отмена расписания (два варианта)
#schedule.clear('daily_import_sheets')
schedule.clear(run_all)